# Azure AI Search s integracijom NVIDIA NIM i LlamaIndex

U ovom bilježniku pokazat ćemo kako iskoristiti NVIDIA-ine AI modele i LlamaIndex za stvaranje moćnog Retrieval-Augmented Generation (RAG) sustava. Koristit ćemo NVIDIA-ove LLM-ove i ugrađivanja, integrirati ih s Azure AI Search kao spremištem vektora te provesti RAG kako bismo poboljšali kvalitetu i učinkovitost pretraživanja.

## Prednosti
- **Skalabilnost**: Iskoristite NVIDIA-ove velike jezične modele i Azure AI Search za skalabilno i učinkovito pretraživanje.
- **Učinkovitost troškova**: Optimizirajte pretraživanje i dohvaćanje pomoću učinkovitog spremišta vektora i hibridnih tehnika pretraživanja.
- **Visoke performanse**: Kombinirajte moćne LLM-ove s vektoriziranim pretraživanjem za brže i preciznije odgovore.
- **Kvaliteta**: Održavajte visoku kvalitetu pretraživanja uz utemeljenje odgovora LLM-a na relevantnim dohvaćenim dokumentima.

## Preduvjeti
- 🐍 Python 3.9 ili noviji
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API ključ za pristup NVIDIA-inim LLM-ovima i ugrađivanjima putem NVIDIA NIM mikroservisa

## Obuhvaćene značajke
- ✅ Integracija NVIDIA LLM-a (koristit ćemo [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA ugrađivanja (koristit ćemo [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Napredni načini dohvaćanja u Azure AI Search
- ✅ Indeksiranje dokumenata pomoću LlamaIndex
- ✅ RAG koristeći Azure AI Search i LlamaIndex s NVIDIA LLM-ovima

Krenimo!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Instalacija i zahtjevi
Kreirajte Python okruženje koristeći Python verziju >3.10.

## Početak!


Da biste započeli, trebate `NVIDIA_API_KEY` za korištenje NVIDIA AI Foundation modela:  
1) Kreirajte besplatan račun na [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Kliknite na željeni model.  
3) Pod Input, odaberite Python karticu, zatim kliknite **Get API Key** i potom **Generate Key**.  
4) Kopirajte i spremite generirani ključ kao NVIDIA_API_KEY. Nakon toga, trebali biste imati pristup endpointima.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Primjer RAG-a koristeći LLM i Ugrađivanje
### 1) Inicijalizacija LLM-a
`llama-index-llms-nvidia`, poznat i kao NVIDIA-ov LLM konektor, omogućuje povezivanje i generiranje iz kompatibilnih modela dostupnih u NVIDIA API katalogu. Pogledajte ovdje popis modela za dovršavanje razgovora: https://build.nvidia.com/search?term=Text-to-Text

Ovdje ćemo koristiti **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Inicijalizirajte Ugrađivanje
`llama-index-embeddings-nvidia`, također poznat kao NVIDIA-ov konektor za ugrađivanja, omogućuje vam povezivanje s kompatibilnim modelima i generiranje iz njih, dostupnim u NVIDIA API katalogu. Odabrali smo `nvidia/nv-embedqa-e5-v5` kao model za ugrađivanje. Ovdje možete pronaći popis modela za tekstualna ugrađivanja: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Stvorite Azure AI Search Vector Store


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Kreirajte upitni mehanizam za postavljanje pitanja o vašim podacima

Ovdje je upit koji koristi čisto pretraživanje vektora u Azure AI Search i temelji odgovor na našem LLM-u (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Evo upita koji koristi hibridno pretraživanje u Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analiza pretraživanja vektora
Odgovor LLM-a točno prenosi ključne ekonomske posljedice navedene u izvornom tekstu vezano za rusko tržište dionica. Konkretno, navodi da je rusko tržište dionica doživjelo značajan pad, izgubivši 40% svoje vrijednosti, te da je trgovanje obustavljeno zbog trenutne situacije. Ovaj odgovor dobro se podudara s informacijama iz izvora, što ukazuje na to da je LLM ispravno identificirao i sažeo relevantne detalje o utjecaju na tržište dionica kao rezultat ruskih postupaka i nametnutih sankcija.

#### Komentar izvora čvorova
Izvorni čvorovi pružaju detaljan prikaz ekonomskih posljedica s kojima se Rusija suočila zbog međunarodnih sankcija. Tekst ističe da je rusko tržište dionica izgubilo 40% svoje vrijednosti, a trgovanje je obustavljeno. Osim toga, spominje i druge ekonomske posljedice, poput devalvacije rublje i šire izolacije ruskog gospodarstva. Odgovor LLM-a učinkovito je sažeo ključne točke iz ovih čvorova, fokusirajući se na utjecaj na tržište dionica, kako je zatraženo u upitu.


Sada, pogledajmo upit u kojem Hibridno pretraživanje ne daje dobro utemeljen odgovor:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hibridno pretraživanje: Analiza odgovora LLM-a
Odgovor LLM-a u primjeru hibridnog pretraživanja ukazuje na to da pruženi kontekst ne navodi točan datum ruske invazije na Ukrajinu. Ovaj odgovor sugerira da LLM koristi informacije dostupne u izvornim dokumentima, ali priznaje nedostatak preciznih detalja u tekstu.

Odgovor je točan u prepoznavanju da kontekst spominje događaje povezane s ruskom agresijom, ali ne navodi specifičan datum invazije. Ovo pokazuje sposobnost LLM-a da razumije pružene informacije, istovremeno prepoznajući praznine u sadržaju. LLM učinkovito potiče korisnika da potraži vanjske izvore ili povijesne zapise za točan datum, pokazujući određenu razinu opreza kada su informacije nepotpune.

### Analiza izvora čvorova
Izvorni čvorovi u primjeru hibridnog pretraživanja sadrže isječke iz govora koji raspravlja o odgovoru SAD-a na ruske akcije u Ukrajini. Ovi čvorovi naglašavaju širi geopolitički utjecaj i korake koje su SAD i njihovi saveznici poduzeli kao odgovor na invaziju, ali ne spominju točan datum invazije. Ovo je u skladu s odgovorom LLM-a, koji ispravno prepoznaje da kontekst ne sadrži informacije o preciznom datumu.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hibridno s ponovnim rangiranjem: Analiza odgovora LLM-a
U primjeru hibridnog pristupa s ponovnim rangiranjem, odgovor LLM-a pruža dodatni kontekst napominjući da se događaj dogodio šest dana prije održavanja govora. To pokazuje da LLM može zaključiti datum invazije na temelju vremena održavanja govora, iako mu je i dalje potrebno znati točan datum govora za preciznost.

Ovaj odgovor demonstrira poboljšanu sposobnost korištenja kontekstualnih tragova za pružanje informativnijeg odgovora. Naglašava prednost ponovnog rangiranja, gdje LLM može pristupiti i prioritizirati relevantnije informacije kako bi dao približniji detalj (npr. datum invazije).

### Analiza izvora čvorova
Izvori čvorova u ovom primjeru uključuju reference na vrijeme ruske invazije, konkretno spominjući da se dogodila šest dana prije govora. Iako točan datum još uvijek nije eksplicitno naveden, čvorovi pružaju vremenski kontekst koji omogućuje LLM-u da pruži nijansiraniji odgovor. Uključivanje ovog detalja pokazuje kako ponovno rangiranje može poboljšati sposobnost LLM-a da izvuče i zaključi informacije iz pruženog konteksta, rezultirajući točnijim i informativnijim odgovorom.


**Napomena:**
U ovom bilježniku koristili smo NVIDIA NIM mikrousluge iz NVIDIA API kataloga. Gore navedeni API-ji, `NVIDIA (llms)`, `NVIDIAEmbedding`, i [Azure AI Search Semantic Hybrid Retrieval (ugrađeno ponovno rangiranje)](https://learn.microsoft.com/azure/search/semantic-search-overview). Napomena, gore navedeni API-ji također podržavaju samostalno hostirane mikrousluge.

**Primjer:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
